# Media

**skrf** supports some basic device simulation based on transmission line models.

* Network creation is accomplished through methods of the `Media` class, which represents a transmission line object for a given medium.
* Once constructed, a `Media` object contains the necessary properties/methods such as:
  * `propagation constant`
  * `characteristic impedance`
  * `line`, `short`, `open`, `resistor`, `inductor`, etc...

In [ ]:
import skrf as rf
from skrf.media import CPW  # Co-Planar Waveguide 
rf.stylely()

freq = rf.Frequency(75,110,101,'ghz')
cpw =  CPW(freq, w=10e-6, s=5e-6, ep_r=10.6)
print(cpw)

To create a transmission line of 100um:

In [ ]:
cpw.line(100*1e-6, name = '100um line')

##  `Media` Object Basics 

Two arguments are common to all media constructors

* `frequency` (required) : a `Frequency` object
*  `z0`   (optional) : `z0` is the port impedance, needed if port impedance is different from the media's characteristic impedance

The instance has  a 10um center conductor, gap of 5um, and substrate with relative permativity of 10.6,

In [ ]:
freq = rf.Frequency(75,110,101,'ghz')
cpw = CPW(freq, w=10e-6, s=5e-6, ep_r=10.6, z0=1)

cpw

* the defining properties of a (single moded) transmission line are it's characteristic impedance and propagation constant.
* These properties return complex `numpy.ndarray`'s
* The *characteristic impedance* is given by a `Z0` (capital Z)

In [ ]:
cpw.Z0[:3] 

* A *port impedance* is also needed when different networks are connected. 
* The *port impedance* is given by `z0` (lower z).
* The *port impedance* is used to compute impedance mismatched if circuits of different port impedance are connected. 

In [ ]:
cpw.z0[:3]  # was set to 1 for the sake of the example

The propagation constant is given by `gamma` 

In [ ]:
cpw.gamma[:3]

## Slab of  Si in Freespace

A plane-wave in freespace from 10-20GHz.

In [ ]:
from skrf.media import Freespace

freq = rf.Frequency(10,20,101,'ghz')
air =  Freespace(freq)
air

In [ ]:
air.z0[:2] # 377ohm baby!

A plane wave in Si:

In [ ]:
si = rf.Freespace(freq, ep_r = 11.2)
si.z0[:3] # ~110ohm


Simulate a 1cm slab of Si in half-space,

In [ ]:
slab = air.thru() ** si.line(1, 'cm') ** air.thru()
slab.plot_s_db(n=0) 

## Rectangular Waveguide

a WR-10 Rectangular Waveguide

In [ ]:
from skrf.media import RectangularWaveguide

freq = rf.Frequency(75,110,101,'ghz')
wg = RectangularWaveguide(freq, a=100*rf.mil, z0=50) # see note below about z0
wg

* The `z0` argument in the Rectangular Waveguide constructor is used to force a specific port impedance.
* This is commonly used to match the port impedance to what a VNA stores in a touchstone file.

Lets compare the propagation constant in waveguide to that of freespace, 

In [ ]:
air = Freespace(freq)

In [ ]:
from matplotlib import pyplot as plt

air.plot(air.gamma.imag, label='Freespace')
wg.plot(wg.gamma.imag, label='WR10')

plt.ylabel('Propagation Constant (rad/m)')
plt.legend()

Because the wave quantities are dynamic they change when the attributes of the media change.

To illustrate, plot the propagation constant of the cpw for various values of substrated permativity,  

In [ ]:
for ep_r in [9,10,11]:
    cpw.ep_r = ep_r
    cpw.frequency.plot(cpw.beta, label='er=%.1f'%ep_r)

plt.xlabel('Frequency [GHz]')
plt.ylabel('Propagation Constant [rad/m]')
plt.legend()


## Network Synthesis

Networks are created through methods of a Media object. To create a 1-port network for a rectangular waveguide short, 

In [ ]:
wg.short(name = 'short') 

Or to create a $90^{\circ}$ section of cpw line, 

In [ ]:
cpw.line(d=90,unit='deg', name='line')

* Simple circuits like :`Media.short` and `open` are ideal short and opens with $\Gamma = -1$ and $\Gamma = 1$
 * they dont take into account sophisticated effects of the discontinuities.

* Eventually, these more complex networks could be implemented with methods specific to a given Media, ie `CPW.cpw_short` , should the need arise...
	

## Building Circuits


By connecting a series of simple circuits, more complex circuits can be 
made. To build a the $90^{\circ}$ delay short, in the 
rectangular waveguide media defined above.

In [ ]:
delay_short = wg.line(d=90,unit='deg') ** wg.short()
delay_short.name = 'delay short'
delay_short

When `Networks` with more than 2 ports need to be connected together, use `rf.connect()` or `Circuit` for more advanced things.

To create a two-port network for a shunted delayed open, you can create an ideal 3-way splitter (a 'tee') and connect the delayed open to one of its ports:

In [ ]:
tee = cpw.tee()
delay_open = cpw.delay_open(40,'deg')
shunt_open = rf.connect(tee,1,delay_open,0)

Adding networks in shunt  is pretty common, so there is a `Media.shunt()` function to do this, 

In [ ]:
cpw.shunt(delay_open)

If a specific circuit is created frequently, it may make sense to 
use a function to create the circuit. This can be done most quickly using `lambda`

In [ ]:
delay_short = lambda d: wg.line(d,'deg')**wg.short()
delay_short(90)

A more useful example may be to create a function for a shunt-stub tuner,
that will work for any media object

In [ ]:
def shunt_stub(med, d0, d1):
    return med.line(d0,'deg')**med.shunt_delay_open(d1,'deg')

shunt_stub(cpw,10,90)

This approach lends itself to design optimization.

## Design Optimization
The abilities of `scipy`'s optimizers can be used to automate network design. In this example, skrf is used to automate the single stub impedance matching network design.

First, we create a 'cost' function which returns something we want to minimize, such as the reflection coefficient magnitude at band center.

Then, one of scipy's minimization algorithms is used to determine the optimal parameters of the stub lengths to minimize this cost.

In [ ]:
from scipy.optimize import fmin

# the load we are trying to match
load = cpw.load(.2+.2j)

# single stub circuit generator function
def shunt_stub(med, d0, d1): 
    return med.line(d0,'deg')**med.shunt_delay_open(d1,'deg')

# define the cost function we want to minimize (this uses sloppy namespace)
def cost(d):
    # prevent negative length lines, returning high cost
    if d[0] <0 or d[1] <0:
        return 1e3 
    return (shunt_stub(cpw,d[0],d[1]) ** load)[100].s_mag.squeeze()

In [ ]:
# initial guess of optimal delay lengths in degrees
d0 = 120,40 # initial guess

#determine the optimal delays
d_opt = fmin(cost,(120,40))

d_opt 